## Lasso Regression

### Setup

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import os
import scipy.stats as stats
import math

!pip install sklearn
from sklearn.linear_model import LassoCV
from sklearn.model_selection import RepeatedKFold
from sklearn.metrics import mean_absolute_error

In [3]:
X_train_full = pd.read_csv("X_train.csv")
X_train_full.head(5)

,High school graduation raw value,Unemployment raw value,Some college raw value,Ratio of population to mental health providers,Median household income raw value,Average Temperature,January Average Temperature,February Average Temperature,March Average Temperature,April Average Temperature,May Average Temperature,June Average Temperature,July Average Temperature,August Average Temperature,September Average Temperature,October Average Temperature,November Average Temperautre,December Average Temperature,Average Precipitation,January Average Precipitation,February Average Precipitation,March Average Precipitation,April Average Precipitation,May Average Precipitation,June Average Precipitation,July Average Precipitation,August Average Precipitation,September Average Precipitation,October Average Precipitation,November Average Precipitation,December Average Precipitation,RUCC
0,0.847000,0.096000,0.568000,4885.000000,60735.0,54.808333,26.6,36.3,44.2,55.8,62.7,72.3,79.2,74.2,65.1,53.8,48.4,39.1,4.859167,1.18,5.42,4.02,11.01,5.75,6.53,2.99,2.16,5.30,3.10,5.95,4.90,1.0
1,0.780000,0.066812,0.547429,954.495385,42945.0,74.058333,59.9,61.5,71.2,72.5,77.2,82.4,84.6,83.6,81.9,76.1,68.5,69.3,4.572500,7.38,2.43,3.46,1.64,7.55,8.57,3.95,6.58,8.25,3.81,0.24,1.01,1.0
2,0.780000,0.069824,0.453978,2573.647059,40994.0,60.825000,38.5,42.9,55.2,59.1,65.9,75.7,79.2,77.6,74.3,63.2,53.7,44.6,3.104167,5.07,5.22,1.65,2.69,3.51,2.22,3.80,5.73,0.96,0.54,1.99,3.87,6.0
3,0.866534,0.059000,0.785187,4209.193548,81586.0,43.408333,16.3,17.3,24.8,39.0,56.3,67.0,72.4,71.4,65.3,48.0,31.4,11.7,2.673333,0.83,1.32,1.79,5.51,5.31,5.99,3.51,1.34,1.62,3.07,0.53,1.26,1.0
4,0.806995,0.102000,0.681151,4158.038961,43863.0,60.991667,46.3,44.2,46.8,61.4,67.8,76.6,79.3,77.1,71.8,62.5,50.0,48.1,4.201667,1.91,4.31,1.74,4.63,3.10,10.77,8.40,5.02,1.72,1.12,2.85,4.85,2.0


In [4]:
Y_train_full = pd.read_csv("Y_train.csv")
Y_train_full.head(5)

,Poor mental health days raw value,Crude Rate
0,4.1,16.6
1,4.2,11.0
2,3.9,24.9
3,2.3,8.7
4,4.0,16.8


In [6]:
identifying_fields = ['State Abbreviation', 'Name', 'Release Year']
with open("factors.txt") as f:
     factors = f.read().splitlines()
with open("outcomes.txt") as f:
     outcomes = f.read().splitlines()

### Lasso regression for predicting Poor Mental Health Days

In [13]:
OUTER_FOLDS = 10
INNER_FOLDS = 10

allMAEs = []
allAlphas = []
allIntercepts = []

avgKFolds = RepeatedKFold(n_splits=OUTER_FOLDS, n_repeats=3, random_state=1)

# outer cross validation is to get the average metrics across random folds in
# order to select best model to use
for train_index, test_index in avgKFolds.split(X_train_full):
  X_train, X_val = X_train_full.iloc[train_index], X_train_full.iloc[test_index]
  Y_train, Y_val = Y_train_full.iloc[train_index], Y_train_full.iloc[test_index]

  '''
  X_train = X_train_full[:math.ceil(X_train_full.shape[0] * 0.8)]
  X_val = X_train_full[-1 * math.floor(X_train_full.shape[0] * 0.2):]

  Y_train = Y_train_full[:math.ceil(Y_train_full.shape[0] * 0.8)]
  Y_val = Y_train_full[-1 * math.floor(Y_train_full.shape[0] * 0.2):]
  '''
  
  # inner cross validation is for the best hyperparameter
  cv = RepeatedKFold(n_splits=INNER_FOLDS, n_repeats=3, random_state=1)
  model = LassoCV(cv=cv, n_jobs=-1)
  model.fit(X_train, Y_train.iloc[:,0])
  print('alpha: %f' % model.alpha_)

  predictions = model.predict(X_val)
  mae = mean_absolute_error(Y_val.iloc[:,0], predictions)
  print(mae)

  coeffs = model.coef_
  important_variables = [i for i, e in enumerate(coeffs) if e != 0]
  for i in important_variables:
      print(factors[i])
      print(coeffs[i])

  allMAEs.append(mae)
  allAlphas.append(model.alpha_)
  allIntercepts.append(model.intercept_)



  

alpha: 4.257934
0.42078332995226914
Ratio of population to mental health providers
2.414122728720153e-06
Median household income raw value
-2.5285453234870454e-05
alpha: 4.229743
0.45673444209229347
Ratio of population to mental health providers
2.9017593552993143e-06
Median household income raw value
-2.5737553204539938e-05
alpha: 4.170363
0.4337646014506057
Ratio of population to mental health providers
2.5793733324226907e-06
Median household income raw value
-2.517477087898092e-05
alpha: 4.228331
0.44095734099906175
Ratio of population to mental health providers
3.2053376060493993e-06
Median household income raw value
-2.534961736223822e-05
alpha: 4.307727
0.4359798083956682
Ratio of population to mental health providers
2.547262746055558e-06
Median household income raw value
-2.5583100462497024e-05
alpha: 4.189706
0.4153409327520425
Ratio of population to mental health providers
2.449128582551217e-06
Median household income raw value
-2.5107636735855138e-05
alpha: 4.215626
0.444061

In [32]:
averageMAE = sum(allMAEs)/OUTER_FOLDS
averageAlpha = sum(allAlphas)/OUTER_FOLDS
averageIntercept = sum(allIntercepts)/OUTER_FOLDS

# Make sure to change this model number for each of the different models. It
# just dictates which row in the csv to put the data. 
model_number = 0

full_metrics_df = pd.read_csv("All_Model_Metrics.csv")
full_metrics_df.loc[model_number] = ["Lasso Regression", averageMAE, averageAlpha, averageIntercept]
full_metrics_df.to_csv('All_Model_Metrics.csv', index=False)